In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
# population is bigger and then take sample 
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('sample_data.csv', index=False)
df.head()

,review,sentiment
711,"Even if it were remotely funny, this mouldy wa...",negative
289,Writer/Director Peter Greenaway cements his ti...,negative
35,A ridiculous comedy given an arms-flailing dir...,negative
677,Set in and near a poor working class town in t...,positive
213,I wasn't really hoping for much when I went to...,negative
